**Numerical Methods Coursework 1**

In [ ]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy.stats as si
import seaborn as sns

# *European and American Options*

First specify tree parameterisation in the Kamrad-Ritchken model

In [ ]:
def KRparam(lamda, sigma, dt, r):
    u = np.exp(lamda * sigma * np.sqrt(dt))
    qu = 1/ (2*lamda**2) + ((r - 0.5*sigma**2) * np.sqrt(dt)) / (2* lamda * sigma)
    qm = 1 - 1/lamda**2
    qd = 1/ (2*lamda**2) - ((r - 0.5*sigma**2) * np.sqrt(dt)) / (2* lamda * sigma)
    if (qu < 0 or qd < 0):
        print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")
        return
    R = np.exp(r * dt)
    return u, qu, qm, qd, R

Specify the pricing algorithm

In [ ]:
def TrinomialTreeEuropean(s0, u, qu, qm, qd, R, N, payoff):
    #initialize the array for stock price and option value
    S = np.zeros((2 * N + 1, N + 1))
    V = np.zeros((2 * N + 1, N + 1))
    for t in range(N+1):
        S[0:2*t+1, t] = s0 * u ** np.arange(t, -t-1, -1)

    #set the final payoff of the option
    V[:, N] = payoff(S[:,N])

    #backward induction
    for t in reversed(range(N)):
        continuation_val = (qu * V[0:2*t+1, t+1] + qm * V[1:2*t+2, t+1] + qd * V[2: 2*t+3, t+1])/R #R = exp(r * dt)
        V[0:2*t+1, t] = continuation_val
    
    return V





In [ ]:
def TrinomialTreeAmerican(s0, u, qu, qm, qd, R, N, payoff):
    #initialize the array for stock price and option value
    S = np.zeros((2 * N + 1, N + 1))
    V = np.zeros((2 * N + 1, N + 1))
    for t in range(N+1):
        S[0:2*t+1, t] = s0 * u ** np.arange(t, -t-1, -1)

    #set the final payoff of the option
    V[:, N] = payoff(S[:,N])

    #backward induction
    for t in reversed(range(N)):
        continuation_val = (qu * V[0:2*t+1, t+1] + qm * V[1:2*t+2, t+1] + qd * V[2: 2*t+3, t+1])/R #R = exp(r * dt)
        V[0:2*t+1, t] = np.maximum(payoff(S[0:2*t+1, t]), continuation_val)
        
    
    return (V, continuation_val[0])

*Implementation of the algorithm for different values of the parameters for European call option*


In part 1. (b), vary $N$, $\lambda$ and $S_0$

In [ ]:
#========1.(b)==========
s0, r, T, sigma, K = 100, 0.01, 1, 0.2, 100 
lamda_val = np.array([1,1.25, 1.5, 1.75])
CallPayoff = lambda x: np.maximum(x - K,0)

def varyNandlamda (s0, N_val, r, T, sigma, K):
    TriTreePrices = np.zeros((len(N_val), len(lamda_val)))
    for i in range(len(lamda_val)):
        lamda = lamda_val[i]
        for j in range(len(N_val)):
            N = N_val[j]
            u, qu, qm, qd, R = KRparam(lamda, sigma, T/N, r)
            V = TrinomialTreeEuropean(s0, u, qu, qm, qd, R, N, CallPayoff)
            TriTreePrices[j,i] = V[0,0]

    European_V = pd.DataFrame(TriTreePrices, columns = [r'$\lambda$=1',r'$\lambda$=1.25',r'$\lambda$=1.5',r'$\lambda$=1.75'], index = N_val, )
    bscall = BS_call(s0, K, T, r, sigma)

    return European_V, bscall

        


Plot the trinomial tree prices against $N$ for different $\lambda$, and compare with the value given by the Black-Scholes formula

In [ ]:
def BS_call(S, K, T, r, sigma):
        
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

#make the plot
def plotagainstN (s0, N_val):
     
    plt.figure()
    European_V, bscall = varyNandlamda( s0, N_val, r, T, sigma, K )
    ax1 = European_V.plot(title = r'Trinomial Tree Prices vs N, $S_0 =$ {}'.format(s0),
                        figsize = (15,10),fontsize = 20, xlim = [0,len(N_val)+1])
    ax1.title.set_size(25)
    ax1.axhline(y = bscall , color = 'k', linewidth = 2, linestyle = '-',label = 'BS call')
    ax1.set_xlabel('Number of Periods N',fontsize = 20)
    ax1.set_ylabel('Option Prices', fontsize = 20)
    ax1.legend(loc=1,fontsize=20)
    plt.savefig('plotoverN{}_{}.pdf'.format(s0,len(N_val)+1))
    plt.show()

plotagainstN(100,np.arange(2,501))
plotagainstN(90,np.arange(2,201))
plotagainstN(100,np.arange(2,201))
plotagainstN(110,np.arange(2,201))


In part 1.(c), fix $N = 500$, $K =100$ and explore the difference between trinomial tree price and BS price for different values of $S_0$ and $\lambda$.

In [ ]:
#========1.(c)=======
r, T, sigma, K ,N =  0.01, 1, 0.2, 100, 500
s0_min = 70
s0_max = 130
s0_val = np.linspace(s0_min,s0_max,71)
price_diff = np.zeros((len(lamda_val), len(s0_val)))

for i in range(len(lamda_val)):
    lamda = lamda_val[i]
    u, qu, qm, qd, R = KRparam(lamda, sigma, T/N, r)
    for j in range(len(s0_val)):
        s0 = s0_val[j]
        V = TrinomialTreeEuropean(s0, u, qu, qm, qd, R, N, CallPayoff)
        bscall = BS_call(s0, K, T, r, sigma)
        price_diff[i,j] = (V[0,0] - bscall)*1000  #we magnify the difference by 10^3 for better accuracy
price_diff_table = pd.DataFrame(price_diff.T, index = s0_val, columns = [r'$\lambda$=1',r'$\lambda$=1.25',r'$\lambda$=1.5',r'$\lambda$=1.75'] )

        
        



In [ ]:
price_diff_table

In [ ]:
abs_diff_table = abs(price_diff_table)
abs_diff = abs(price_diff_table).mean(axis = 0)
abs_diff

In [ ]:
relative_mag = abs_diff_table.iloc[:,1] > abs_diff_table.iloc[:,2]
relative_mag.mean()


*Implementation of the algorithm for American Call Options*

In part 2.(b), compare the time-zero value of time-zero value of an American call option and its intrinsic value agaiinst different initial stock prices $S_0$.

In [ ]:
r, T, sigma, K ,N, lamda =  0.01, 1, 0.2, 100, 500, 1.25
u, qu, qm, qd, R = KRparam(lamda, sigma, T/N, r)
s0_min = 1
s0_max = 150
s0_val = np.linspace(s0_min,s0_max,151)

American_call_intrinsic = CallPayoff(s0_val)
American_call_V = np.array([TrinomialTreeAmerican(s0, u, qu, qm, qd, R, N, CallPayoff)[0][0,0] for s0 in s0_val])
data = np.column_stack([American_call_intrinsic, American_call_V])
early_exercise_data_call = pd.DataFrame(data= data, index = s0_val, columns = ['American call intrinsic value', 'American call option value'])

fig, ax = plt.subplots(figsize = (20,10))
ax.plot(s0_val, early_exercise_data_call['American call option value'], color = 'b', label = 'Value from Trinomial Tree Model')
ax.plot(s0_val, early_exercise_data_call['American call intrinsic value'], color = 'r', label = "Intrinsic Value")
ax.set_xlabel('Initial Stock Price', fontsize = 20)
ax.set_ylabel('Option Values', fontsize = 20)
ax.set_title('Time-zero values of American call option', fontsize = 25)
plt.xticks(np.arange(0,160,10),fontsize = 20)
plt.yticks(fontsize = 20)
ax.legend(loc = 2, fontsize = 20)
plt.savefig('american_call.pdf')


In [ ]:
100/u**500


In part 2.(c), we work with American put options and explore the early exercise strategy when volatility $\sigma$ and interest rate $r$ change.


In [ ]:
r, T, sigma, K ,N, lamda =  0.01, 1, 0.2, 100, 500, 1.25
u, qu, qm, qd, R = KRparam(lamda, sigma, T/N, r)
s0_min = 1
s0_max = 150
s0_val = np.linspace(s0_min,s0_max,200)

PutPayoff = lambda x : np.maximum(K - x, 0)
A_Put_intrinsic = PutPayoff(s0_val)
A_Put_V = np.array([TrinomialTreeAmerican(s0, u, qu, qm, qd, R, N, PutPayoff)[0][0,0] for s0 in s0_val])
data = np.column_stack([A_Put_intrinsic.T, A_Put_V.T])
early_exercise_data_put = pd.DataFrame(data= data, index = s0_val, columns = ['American put intrinsic value', 'American put option value'])
fig, ax = plt.subplots(figsize = (20,10))
ax.plot(s0_val, early_exercise_data_put['American put option value'], color = 'b', label = 'Value from Trinomial Tree Model')
ax.plot(s0_val, early_exercise_data_put['American put intrinsic value'], color = 'r', label = "Intrinsic Value")
ax.set_xlabel('Initial Stock Price', fontsize = 20)
ax.set_ylabel('Option Values', fontsize = 20)
ax.set_title('Time-zero values of American put option', fontsize = 25)
plt.xticks(np.arange(0,160,10),fontsize = 20)
plt.yticks(fontsize = 20)
ax.legend(fontsize = 20)

plt.savefig("american_put_intrinsic_v_trinomial.eps")

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
plt.plot(early_exercise_data_put[0:100]['American put option value'] - early_exercise_data_put[0:100]['American put intrinsic value'])
ax.set_xlabel('Initial Stock Price', fontsize = 20)
ax.set_ylabel('Option Value - Intrinsic Value', fontsize = 20)
ax.set_title('Difference between time-zero values of American put option', fontsize = 25)
plt.xticks(np.arange(0,110,10),fontsize = 20)
plt.yticks(fontsize = 20)
ax.axhline(0, color = 'black')
plt.savefig("ValueDifferencePut.eps")

In [ ]:
def find_early_exercise_cutoff(r, T, sigma, K ,N, lamda):
    u, qu, qm, qd, R = KRparam(lamda, sigma, T/N, r)
    s0_min = 1
    s0_max = 100
    s0_val = np.linspace(s0_min,s0_max,200)

    PutPayoff = lambda x : np.maximum(K - x, 0)
    A_Put_intrinsic = PutPayoff(s0_val)
    A_Put_V = np.array([TrinomialTreeAmerican(s0, u, qu, qm, qd, R, N, PutPayoff)[0][0,0] for s0 in s0_val])
    
    difference = pd.Series(A_Put_V - A_Put_intrinsic, index=s0_val)

    if (difference.size == 0):
        return -1
    return difference[difference > 0].index[0]
    

In [ ]:

find_early_exercise_cutoff(0.01, T, 0.2, K ,N, lamda)

In [ ]:
sigma_range = np.linspace(0.01, 0.3, 30)
r_range = np.linspace(0, 0.1, 101)
cutoff_points = np.zeros([101, 30])

for s_index, s in enumerate(sigma_range):
    print(s)
    for r_index, r_temp in enumerate(r_range):
        cutoff = find_early_exercise_cutoff(r_temp, T, s, K ,N, lamda)
        cutoff_points[r_index, s_index] = cutoff

cutoff_df = pd.DataFrame(cutoff_points)
cutoff_df.to_pickle("cutoff_dataframe")

In [ ]:
cutoff_df = pd.read_pickle("cutoff_dataframe")
cutoff_df.index=np.round(r_range,3)
cutoff_df.columns = np.round(sigma_range,2)

In [ ]:
ax = sns.heatmap(cutoff_df[0.001:], yticklabels= 10)
ax.set_ylim([0,101])
ax.set_xlabel("$\sigma$", fontsize=13)
ax.set_ylabel("$r$", fontsize=13)
plt.savefig("heatmap.eps", bbox_inches = 'tight',pad_inches = 0.07)



Question 3)

Firstly we define the Kamrad-Ritchken model.

In [ ]:

def create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma, N, lamda):
    #model parameters
    
    u, qu, qm, qd, R = KRparam(lamda, sigma, dt, r)

    if (qu < 0 or qd < 0):
        print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")
        return

    # Set up the tree via S[n,k] = s0 * u^(k-n)
    prices = np.zeros((N+1, 2*N+1))
    for t in range(N+1):
        prices[t, 0:2*t+1] = s0 * u**np.arange(t, -t-1, -1)

    return (prices, np.array([qd, qm, qu]), u)

    
        

We create an array of prices for $A^n_j$ noticing that these prices have no dependence on $n$.

In [ ]:
def create_auxillary_values(s0, N, u):
    indices = np.arange(0,N+1)
    AValues = s0*u**indices
    return AValues

Next we define the funtion for generating the prices by backward induction

In [ ]:
def generate_lookback_prices_optimized(s0, r, dt, sigma, N, lamda, american = False):
    Nfloor_div_2 = N//2     
    prices, probabilities, u = create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma,  N, lamda)
    auxillary_values = create_auxillary_values(s0, N, u)
            
    # Option values matrix indexed by time, k, j
    option_values = np.zeros([N+1, 2*N+1, Nfloor_div_2+1])

    # Terminal condition: option values given by terminal payoff

    terminal_stock_price = prices[N]
    k_range = np.arange(0,2*N + 1)
    j_range = np.arange(0, Nfloor_div_2+1)

    #find x such that A^N_{k,j} = S_0u**x indexed by k, j
    #Note this can produce negative indexes, but these represent impossible situations
    #so they will not impact any resulting prices.
    auxiliary_variable_index = N-((k_range+1)//2)[:,np.newaxis] - Nfloor_div_2 + j_range[np.newaxis, :]
    option_values[N] = np.maximum(np.take_along_axis(auxillary_values[np.newaxis,:], auxiliary_variable_index, axis = 1) - terminal_stock_price[:,np.newaxis], np.zeros(1)[:,np.newaxis])


    # Backward induction
    # Loop in time
    for n in reversed(range(N)):
        #generate the possible values of k_new indexed by current k, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        current_k = np.arange(0,2*n+1)
        k_new = np.column_stack([current_k+2, current_k+1, current_k])
        floor_k_new_div_2 = k_new//2
        ceil_k_div_2 = (current_k+1)//2
        floor_k_div_2 = current_k//2
        j_range = np.arange(Nfloor_div_2-n//2, Nfloor_div_2+1)
        alpha = (k_new[:,np.newaxis, :]+1)//2 - 1 - ceil_k_div_2[:, np.newaxis, np.newaxis] + j_range[np.newaxis,:,np.newaxis]
        beta = Nfloor_div_2 - floor_k_new_div_2[:,np.newaxis, :]

        #Use the shooting function to generate the possible values of jnew indexed by current k, current j, stock movement where for the stock movement 0 is down, 1 is no movement, 2 is up.
        phi_result = np.maximum(alpha, beta)

        #calculate the option prices using backward induction
        v_down = np.take_along_axis(option_values[n+1, current_k+2, :], phi_result[:,:,0], axis = 1)
        v_mid = np.take_along_axis(option_values[n+1, current_k + 1, :], phi_result[:,:,1], axis = 1)
        v_up = np.take_along_axis(option_values[n+1, current_k, :], phi_result[:,:,2], axis = 1)
        discounted_continuation_values = np.exp(-1*r*dt)*(probabilities[0]* v_down +  probabilities[1]* v_mid + probabilities[2]* v_up)

        if american:

            #find x such that A^n_{k,j} = S_0u**x indexed again by k, j
            auxiliary_variable_index = n-((current_k+1)//2)[:,np.newaxis] - Nfloor_div_2 + j_range[np.newaxis, :]
            
            current_stock_price = prices[n]
            exercise_values = np.maximum(np.take_along_axis(auxillary_values[np.newaxis,:], auxiliary_variable_index, axis = 1) - current_stock_price[current_k, np.newaxis], np.zeros(1)[:,np.newaxis])
            option_values[n][0:2*n+1,j_range] = np.maximum(exercise_values, discounted_continuation_values)
        else:
            option_values[n][0:2*n+1,j_range] =  discounted_continuation_values


    #print the fair price for the initial time
    return option_values[0,0,Nfloor_div_2]

In [ ]:
generate_lookback_prices_optimized(100, 0, 1, 0.1, 3, 1.25, False)

We include a less optimised version for reference, the output is the same in both cases

In [ ]:
def generate_lookback_prices(s0, r, dt, sigma, N, lamda, american = False):
    Ndiv2 = N//2
    prices, probabilities, u = create_trinomial_model_kamrad_ritchken(s0, r, dt, sigma,  N, lamda)
            
    # Option values matrix
    OptPx = np.zeros([N+1, 2*N+1, Ndiv2+1])
    
    
    # Terminal condition: option values given by terminal payoff
    for k in range(2*N+1):
        j_range = np.arange(0, Ndiv2+1)
        OptPx[N, k, j_range] = np.maximum(s0 * u **(N - (k+1)//2 -N//2 + j_range) - prices[N, k], 0)

        
    for n in reversed(range(N)):
        # Loop across each stock price index k
        for k in range(2*n+1):
            
            j_range = np.arange(0, N//2+1)
            
            alpha_up = (k+1)//2 - 1 - (k+1)//2 + j_range
            alpha_mid = (k+2)//2 - 1 - (k+1)//2 + j_range
            alpha_down = (k+3)//2 - 1 - (k+1)//2 + j_range

            beta_up = N//2-k//2
            beta_mid = N//2-(k+1)//2
            beta_down = N//2-(k+2)//2

            phi_up = np.maximum(alpha_up, beta_up)  
            phi_mid = np.maximum(alpha_mid, beta_mid)
            phi_down = np.maximum(alpha_down, beta_down)
            
            V_up = OptPx[n+1, k, phi_up]
            V_mid = OptPx[n+1, k+1, phi_mid]
            V_down = OptPx[n+1, k+2, phi_down]
            
            
            
            # the backward induction step
            if american:
                OptPx[n, k, j_range] = np.maximum(np.maximum(s0 *u **(n - (k+1)//2 -N//2 + j_range) - prices[n, k], 0), np.exp(-1*r*dt)*(probabilities[2]*V_up + probabilities[1]*V_mid + probabilities[0]*V_down))
            else:
                OptPx[n,k,j_range] = np.exp(-1*r*dt)*(probabilities[2]*V_up + probabilities[1]*V_mid + probabilities[0]*V_down)

            
        
    return OptPx[0,0,N//2]

*Finally let's run the code to obtain the price

In [ ]:
s0 = 100
r = 0.01

sigma = 0.2
N = np.array([1, 5, 10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]) 
dt = 1/N
lamda = 1.25


In [ ]:
generate_lookback_prices_optimized(s0, r, 1/700, sigma, 700, lamda, False)
performance_data_optimized = pd.DataFrame()

iterations = 10
for american in [True, False]:
    rows = []
    index = []
    for n, d_t in zip(reversed(N),reversed(dt)):
        total_time = 0
        for x in range(iterations):
            local_iterations = 0
            try:
                start = time.perf_counter_ns()
            
                price = generate_lookback_prices_optimized(s0, r, d_t, sigma, n, lamda, american)
                stop = time.perf_counter_ns()
                total_time += stop-start
                local_iterations += 1
            except Exception:
                print("faied for size, iteration: ", n," ", x)
        if local_iterations > 0:
            rows.append([price, total_time/local_iterations])
            index.append((american, n))
    
    newDataFrame = pd.DataFrame(reversed(rows), columns=[ "Value produced", "Running time"], index = pd.MultiIndex.from_tuples(reversed(index), names=["American", "Number of iterations"]))
    performance_data_optimized = performance_data_optimized.append(newDataFrame)

performance_data_optimized


In [ ]:

performance_data_optimized["Running time"]=performance_data_optimized["Running time"]/10**9
performance_data_optimized.to_pickle("perf_data_optimized")


In [ ]:
generate_lookback_prices(s0, r, 1/700, sigma, 700, lamda, False)

performance_data_simple = pd.DataFrame()

iterations = 10
for american in [True, False]:
    rows = []
    index = []
    for n, d_t in zip(reversed(N),reversed(dt)):
        total_time = 0
        for x in range(iterations):
            local_iterations = 0
            try:
                start = time.perf_counter_ns()
            
                price = generate_lookback_prices(s0, r, d_t, sigma, n, lamda, american)
                stop = time.perf_counter_ns()
                total_time += stop-start
                local_iterations += 1
            except Exception:
                print("faied for size, iteration: ", n," ", x)
        if local_iterations > 0:
            rows.append([price, total_time/local_iterations])
            index.append((american, n))
    
    newDataFrame = pd.DataFrame(reversed(rows), columns=[ "Value produced", "Running time"], index = pd.MultiIndex.from_tuples(reversed(index), names=["American", "Number of iterations"]))
    performance_data_simple = performance_data_simple.append(newDataFrame)

performance_data_simple

In [ ]:
american_runtime = performance_data_optimized.loc[True,:]
european_runtime = performance_data_optimized.loc[False,:]
plt.scatter(american_runtime.index, american_runtime["Running time"], label = "American option")
plt.scatter(european_runtime.index, european_runtime["Running time"], label = "European option")
parameters_american, _ = curve_fit(lambda x,a,b,c: a*x**3 + b*x**2 + c*x, american_runtime.index, american_runtime["Running time"])
parameters_european, _ = curve_fit(lambda x,a,b,c: a*x**3 + b*x**2 + c*x, european_runtime.index, european_runtime["Running time"])
x = np.linspace(0,N[-1])
y_american = parameters_american[0]*x**3 + parameters_american[1]*x**2 + parameters_american[2]*x
y_european = parameters_european[0]*x**3 + parameters_european[1]*x**2 + parameters_european[2]*x
plt.xlabel("Number of iterations")
plt.ylabel("Algorithm running time (s)")
plt.legend()

plt.plot(
    x,
    y_american)
plt.plot(
x,
y_european)

plt.savefig("algo_runtime.eps")


In [ ]:
american_value = performance_data_optimized.loc[True,:]
european_value = performance_data_optimized.loc[False,:]
plt.plot(american_value.index, american_value["Value produced"], label = "American option")
plt.scatter(american_value.index, american_value["Value produced"])
plt.plot(european_runtime.index, european_value["Value produced"], label = "European option")
plt.scatter(european_runtime.index, european_value["Value produced"])
x = np.linspace(0,N[-1])

plt.xlabel("Number of iterations (N)")
plt.ylabel("Value estimated by algorithm")
plt.legend()
#plt.ylim(3,8)
#plt.yticks(np.arange(3,8.5,0.5))
plt.savefig("algoConvergence.eps")